In [28]:
from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

In [38]:
import pandas as pd
import numpy as np
import os
from numpy import random as nprandom
import random
import time
from multiprocessing import Pool
import sys
from queue import Queue
from threading import Thread
import multiprocessing
import tensorflow as tf

featuresPath = os.path.join('..', 'features')
lookback = 10
winNoCols = ['WinNo'+str(i) for i in range(1,21)]

df = pd.read_csv(os.path.join(featuresPath, 'features.csv'))
df.reset_index(drop=True, inplace=True)
assert not np.any(np.isnan(df.values))               # make sure we do not have nan values
df.head()

,id,WinNo1,WinNo2,WinNo3,WinNo4,WinNo5,WinNo6,WinNo7,WinNo8,WinNo9,...,e_sinId%43,e_cosId%43,sq_sinId%43,sq_cosId%43,sinId%47,cosId%47,e_sinId%47,e_cosId%47,sq_sinId%47,sq_cosId%47
0,495689,6,11,17,22,25,26,30,35,42,...,0.443397,0.558859,0.661440,0.338560,-0.451010,-0.892519,0.636984,0.409623,0.203410,0.796590
1,67269,9,17,21,23,24,26,32,34,35,...,1.842593,0.453166,0.373533,0.626467,0.999442,-0.033415,2.716764,0.967137,0.998884,0.001117
2,213192,5,7,14,15,23,27,34,36,39,...,0.749687,2.605437,0.083001,0.916999,0.000000,1.000000,1.000000,2.718282,0.000000,1.000000
3,145080,1,3,12,14,22,24,25,30,34,...,0.749687,2.605437,0.083001,0.916999,-0.933181,0.359408,0.393301,1.432481,0.870826,0.129174
4,230127,1,8,9,12,21,25,26,28,29,...,0.380038,1.287797,0.936025,0.063975,0.907091,-0.420935,2.477106,0.656433,0.822814,0.177186


In [39]:
df2 = df[winNoCols]
df2.describe()

,WinNo1,WinNo2,WinNo3,WinNo4,WinNo5,WinNo6,WinNo7,WinNo8,WinNo9,WinNo10,WinNo11,WinNo12,WinNo13,WinNo14,WinNo15,WinNo16,WinNo17,WinNo18,WinNo19,WinNo20
count,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.000000,2508.00000,2508.000000
mean,3.865630,7.713716,11.508772,15.376794,19.177432,23.042265,26.901914,30.827751,34.627990,38.523923,42.506778,46.320973,50.214514,54.091308,57.970893,61.756380,65.617624,69.494817,73.34689,77.110447
std,3.188389,4.409370,5.230118,5.848606,6.294591,6.701913,6.996434,7.205059,7.373989,7.398497,7.394974,7.339524,7.210953,7.015627,6.719785,6.370371,5.876888,5.160717,4.36307,3.196678
min,1.000000,2.000000,3.000000,4.000000,5.000000,6.000000,10.000000,12.000000,13.000000,15.000000,17.000000,19.000000,28.000000,30.000000,32.000000,33.000000,40.000000,42.000000,47.00000,52.000000
25%,2.000000,4.000000,8.000000,11.000000,15.000000,18.000000,22.000000,26.000000,29.000000,34.000000,37.000000,41.000000,45.000000,49.000000,54.000000,58.000000,62.000000,66.000000,71.00000,76.000000
50%,3.000000,7.000000,11.000000,15.000000,19.000000,23.000000,26.000000,31.000000,34.000000,39.000000,43.000000,46.000000,51.000000,55.000000,59.000000,62.000000,66.000000,70.000000,74.00000,78.000000
75%,5.000000,10.000000,15.000000,19.000000,23.000000,27.000000,32.000000,36.000000,40.000000,43.000000,48.000000,52.000000,55.000000,59.000000,63.000000,66.000000,70.000000,73.000000,77.00000,79.000000
max,29.000000,31.000000,42.000000,43.000000,45.000000,47.000000,52.000000,56.000000,57.000000,60.000000,64.000000,66.000000,69.000000,73.000000,74.000000,75.000000,77.000000,78.000000,79.00000,80.000000


In [40]:
otherCols = [col for col in df.columns if col not in winNoCols and col!='id']
df3 = df[otherCols]
values = df3.values
print("Min value: ", values.min())
print("Max value: ", values.max())

Min value:  -1.0
Max value:  2.7182818


# Prepare Dictionaries

In [5]:
numWinNumbers = 20
contFDict = {}
seqFDict = {}
winNumbersTDict = {}

start = time.time()
print("Creating dictionaries for batchPrep")
data = df.values

def getPastWinningNumbers(drawId): 
    return data[drawId-lookback:drawId, 1:numWinNumbers+1]
    
for example in [x for x in df.iloc[lookback:].values]:     # values start from id=lookback+1 !
    drawId = int(example[0])
    winNumbers = example[1:numWinNumbers+1]
    seqFDict[drawId] = getPastWinningNumbers(drawId)       # get previous 'lookback' times winning numbers
    contFDict[drawId] = example[numWinNumbers+1:]                       # continuous features
    winNumbersTDict[drawId] = winNumbers                   # the targets (12 numbers)
    
end = time.time()
elapsed = round(end-start,3)
print("Elapsed time: "+str(elapsed)+" seconds!")

Creating dictionaries for batchPrep
Elapsed time: 1.862 seconds!


# Batch Prep

In [6]:
batchSize = 64
batchesNo = 100
evalInterval = int(batchesNo/5)
drawIds = set(winNumbersTDict.keys())

def batchPrep():
    batchDrawIds = random.sample(drawIds, batchSize)
    contFeatures = np.array([contFDict[x] for x in batchDrawIds])
    seqFeatures = np.array([seqFDict[x] for x in batchDrawIds])
    targets = np.array([winNumbersTDict[x] for x in batchDrawIds])
    return contFeatures, seqFeatures, targets
    
start = time.time()
print("Running a batchprep simulation of size "+str(batchesNo))
for i in range(batchesNo):
    contFeatures, seqFeatures, targets = batchPrep()
    if i>0 and i%evalInterval==0:
        print("Eval time!")
        
end = time.time()
elapsed = round(end-start,3)
print("Elapsed time: "+str(elapsed)+" seconds!")
print("Batch speed: "+str(batchesNo/elapsed)+" batches/sec")

Running a batchprep simulation of size 100
Eval time!
Eval time!
Eval time!
Eval time!
Elapsed time: 1.056 seconds!
Batch speed: 94.69696969696969 batches/sec


In [7]:
print(contFeatures.shape)       # batchsize, features
print(seqFeatures.shape)        # batchsize, timesteps (lookback), features
print(targets.shape)            # batchsize, targetsize

(64, 132)
(64, 10, 20)
(64, 20)


# 1st attempt: only feed features (no lstm)

## 1.2 Singlethreaded batchPrep

In [8]:
inputSize = contFeatures.shape[1]
print(inputSize)
targetSize = targets.shape[1]
print(targetSize)

132
20


In [9]:
# define weights and biases

tf.reset_default_graph()

biases = {
    'b1': tf.get_variable('b1', shape=(256,), initializer = tf.random_normal_initializer(0, 0.05)),
    'b2': tf.get_variable('b2', shape=(128,), initializer = tf.random_normal_initializer(0, 0.05)),
    'b3': tf.get_variable('b3', shape=(64,), initializer = tf.random_normal_initializer(0, 0.05)),
    'b4': tf.get_variable('b4', shape=(targetSize,), initializer = tf.random_normal_initializer(0, 0.05))
}

weights = {
    'w1': tf.get_variable('w1', shape=(inputSize, 256), initializer = tf.random_normal_initializer(0, 0.05)),
    'w2': tf.get_variable('w2', shape=(256, 128), initializer = tf.random_normal_initializer(0, 0.05)),
    'w3': tf.get_variable('w3', shape=(128, 64), initializer = tf.random_normal_initializer(0, 0.05)),
    'w4': tf.get_variable('w4', shape=(64, targetSize), initializer = tf.random_normal_initializer(0, 0.05))
}

In [10]:
# define TF layer functions and loss

def squared_loss(targets, prediction):
    return tf.reduce_mean(tf.math.abs(targets - prediction))

def fcLayer(x, w, b, activation=None):
    result = tf.matmul(x, w) + b
    if activation == 'relu':
        result = tf.nn.leaky_relu(result)         # normal will cut off most values
    if activation == 'softmax':
        result = tf.nn.softmax(result,axis=1)
    return result

In [11]:
# Graph operations

featuresInput = tf.placeholder(name="featuresInput", dtype=tf.float32, shape=(None, inputSize))
seqInput = ...
targetsOutput = tf.placeholder(name="targetsOutput", dtype=tf.float32, shape=(None, targetSize))

with tf.variable_scope("contFeatures"):
    x1 = fcLayer(featuresInput, weights['w1'], biases['b1'], 'relu')
    x2 = fcLayer(x1, weights['w2'], biases['b2'], 'relu')
    x3 = fcLayer(x2, weights['w3'], biases['b3'], 'relu')
    output = fcLayer(x3, weights['w4'], biases['b4']) 
# with tf.variable_scope("seqFeatures"):
#     y1 = 
    #y2 = 
# Concatenate
# Add a few more fcLayers
#with tf.variable_scope("jointFeatures"):
     

loss = squared_loss(targetsOutput, output)
optim = tf.train.AdamOptimizer(0.002, 0.5)
train = optim.minimize(loss)

In [12]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = False
config.allow_soft_placement = True
saver = tf.train.Saver()

In [14]:
def batchPrep():
    batchDrawIds = random.sample(drawIds, batchSize)
    contFeatures = np.array([contFDict[x] for x in batchDrawIds])
    seqFeatures = np.array([seqFDict[x] for x in batchDrawIds])
    targets = np.array([winNumbersTDict[x] for x in batchDrawIds])
    #targets2 = np.ones((64,20))
    return contFeatures, seqFeatures, targets

In [15]:
sess = tf.InteractiveSession(config=config)
sess.run(tf.global_variables_initializer())

In [16]:
from sklearn.metrics import mean_absolute_error
import itertools

random.seed(30)
evalDrawIds = random.sample(set(winNumbersTDict.keys()), int(len(df)*0.002))
testDrawIds = random.sample(set(winNumbersTDict.keys()), int(len(df)*0.002))
testDrawIds[:5]

[479358, 351265, 600595, 791166, 431533]

In [25]:
def processResults(targets, predictions):
    for i in range(len(targets)):
    #for predRow, targetRow in zip(predictions, targets):                   # iterate through rows
        targetSet = set(targets[i,:])
        predSet = set(predictions[i,:])
        #print(len(targetSet), len(predSet))
        for groupSize in range(10, maxPlayableNumbers+1):
            count=0
            recGroups = list(itertools.combinations(predSet, groupSize))
            #print(len(recGroups))
            for group in recGroups:
                if set(group).issubset(targetSet):      #set(group) <= targetSet:
                    count+=1
                    print("Count>0!")
                
start = time.time()

for i in range(20):
    processResults(targets, preds)
    
end = time.time()
elapsed = round(end-start,3)
print("Elapsed time: ",elapsed," seconds")

Elapsed time:  10.071  seconds


In [26]:
def processResults(targets, predictions):
    for predRow, targetRow in zip(predictions, targets):                   # iterate through rows
        targetSet = set(targetRow.flatten())
        predSet = set(predRow.flatten())
        print(len(targetSet), len(predSet))
        for groupSize in range(10, maxPlayableNumbers+1):
            count=0
            recGroups = list(itertools.combinations(predSet, groupSize))
            for group in recGroups:
                if group in targetSet:
                    count+=1
                    print("Count>0!")
            #print("\tGroupsize: ",groupSize, " valid Combinations: ", count," out of ",len(recGroups))


def runInference(runType):
    if runType=='eval':
        contFeatures = np.array([contFDict[x] for x in evalDrawIds])
        seqFeatures = np.array([seqFDict[x] for x in evalDrawIds])
        targets = np.array([winNumbersTDict[x] for x in evalDrawIds])
    elif runType=='test':
        contFeatures = np.array([contFDict[x] for x in testDrawIds])
        seqFeatures = np.array([seqFDict[x] for x in testDrawIds])
        targets = np.array([winNumbersTDict[x] for x in testDrawIds])
    else:
        raise Exception('runInference() runType should be in "eval" or "test", '+str(runType)+" provided")
    
    fd = {featuresInput: contFeatures, targetsOutput: targets}
    _, l, out = sess.run(fetches=[train, loss, output], feed_dict=fd)
    predictions = np.round(out)
    sqLoss = mean_absolute_error(targets, predictions)# ((targets - predictions)**2).mean()        # mean average error
    if runType=='eval':
        print("\tEval MAE: ", round(sqLoss,3))
    else:
        print("\tTest MAE: ", round(sqLoss,3))
        #processResults(targets, predictions)
        
    return predictions

In [27]:
batchSize = 100
batchesNo = 800

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("Training:")
    for i in range(batchesNo):
        contFeatures, seqFeatures, targets = batchPrep()
        #print("\tInput: ", contFeatures[0:1,0:5], ", targets:", targets[:1,:5])
        fd = {featuresInput: contFeatures, targetsOutput: targets}
        _, l, out, xx1, fi = sess.run(fetches=[train, loss, output, x1, featuresInput], feed_dict=fd)
        if i%(batchesNo/10)==0:
            print("\tBatch ",i,": Train loss: ", l)
        #print('\n\tOutput: ', out[1:3,1:3])
        #print("\tFeaturesInput: ", fi[0:1,0:5])
        #print("\tx1: ", xx1)
        if i>0 and i%100==0:
            runInference(runType='eval')
    print("Testing:")
    runInference(runType='test')        

Training:
	Batch  0 : Train loss:  40.87414
	Batch  80 : Train loss:  5.0129485
	Eval MAE:  4.776
	Batch  160 : Train loss:  5.6232667
	Eval MAE:  4.857
	Batch  240 : Train loss:  5.6753416
	Eval MAE:  5.047
	Batch  320 : Train loss:  4.7807775
	Batch  400 : Train loss:  4.746998
	Eval MAE:  4.881
	Batch  480 : Train loss:  5.0752554
	Eval MAE:  4.844
	Batch  560 : Train loss:  4.8204107
	Eval MAE:  4.872
	Batch  640 : Train loss:  5.0155206
	Eval MAE:  5.343
	Batch  720 : Train loss:  5.7913837
Testing:
	Test MAE:  5.061


In [182]:
# test

batchSize = 64
batchesNo = 1000
evalInterval = int(batchesNo/5)
drawIds = set(winNumbersTDict.keys())

def batchPrep():
    batchDrawIds = random.sample(drawIds, batchSize)
    contFeatures = np.array([contFDict[x] for x in batchDrawIds])
    seqFeatures = np.array([seqFDict[x] for x in batchDrawIds])
    targets = np.array([winNumbersTDict[x] for x in batchDrawIds])
    targets2 = np.ones((64,12))
    return contFeatures, seqFeatures, targets2
    
start = time.time()
print("Running a batchprep simulation of size "+str(batchesNo))
for i in range(batchesNo):
    contFeatures, seqFeatures, targets = batchPrep()
    if i>0 and i%evalInterval==0:
        print("Eval time!")
        
end = time.time()
elapsed = round(end-start,3)
print("Elapsed time: "+str(elapsed)+" seconds!")
print("Batch speed: "+str(batchesNo/elapsed)+" batches/sec")

Running a batchprep simulation of size 1000
Eval time!
Eval time!
Eval time!
Eval time!
Elapsed time: 9.404 seconds!
Batch speed: 106.33772862611654 batches/sec


## 1.2 Multithreaded Approach

In [8]:
def batchPrep():
    """
    Prepares a batch and puts it in the queue for the other thread to process 
    """
    for i in range(batchesNo): 
        batchDrawIds = random.sample(drawIds, batchSize)                # prepares batch
        contFeatures = np.array([contFDict[x] for x in batchDrawIds])
        seqFeatures = np.array([seqFDict[x] for x in batchDrawIds])
        targets = np.array([winNumbersTDict[x] for x in batchDrawIds])
        q.put((contFeatures, seqFeatures, targets))                     # put it in the queue
    
def trainBatch(BatchTuple):
    """
    Feeds a batch to the model for training and prints Loss
    Args: 
      BatchTuple: the required input for the model corresponding to one batc/h 
    Returns: Sucess Code
    """
    # TF model train
    # print loss
    return 0

def runEvaluation():
    """
    Runs every batchesNo/5 steps to evaluate performance on eval set and prints loss
    Returns: Sucess Code
    """
    return 0

def trainModel(queue, batchesNo):
    """
    Spawns a thread to produce batches and main thread feeds them to the model
    Args: 
      queue: in OOP design, it will be self.queue
      batchesNo: in OOP design, it will be self.batchesNo
    """     
    t = Thread(target=batchPrep)
    t.daemon = True
    t.start()
    for i in range(batchesNo):  
        batchtuple = queue.get()
        trainBatch(batchtuple)
        if i>0 and i%500==0:
            print("Running Evaluation!")
            runEvaluation()
        queue.task_done()
    try:
        queue.join()                               # Join all Threads
    except KeyboardInterrupt:
        sys.exit(1)    

simNo = 1                      # number of simulations
start = time.time()
for i in range(simNo):            
    q = Queue(maxsize=5)       # batch queue
    batchesNo = 1000
    trainModel(q, batchesNo)                        

end = time.time()
elapsed = round(end-start,3)
print("Elapsed time: "+str(elapsed)+" seconds!")
if elapsed>0:
    print("Batch speed: "+str(simNo*batchesNo/elapsed)+" batches/sec")

Running Evaluation!
Elapsed time: 9.071 seconds!
Batch speed: 110.24142872891633 batches/sec
